In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
import json
from langchain.schema import Document

with open('./data/gold_doc.json', 'r', encoding='utf-8') as f:
    documents = json.load(f)

documents

{'breeds': [{'title': '골든 리트리버',
   'sections': [{'topic': '견종백과',
     'subtopics': [{'title': '성격',
       'content': ['너무 착하고 사람을 좋아해서 문제가 되기도 함.',
        '도둑도 반기는 경우 있음.',
        '침을 많이 흘리는 특성.']},
      {'title': '물에 대한 본능',
       'content': ['조상들의 영향으로 물을 좋아함.', '연못이 보이면 들어가버릴 정도.']},
      {'title': '장점',
       'content': ['머리가 좋고 사람과 함께 있는 것을 좋아함.', '기본 명령은 자연스럽게 익힘.']},
      {'title': '안내견 가능성',
       'content': ['모든 리트리버가 가능한 것은 아님.', '안내견은 특별히 선별된 개체들.']},
      {'title': '질병',
       'content': ['인브리딩으로 인한 유전적 문제, 암 발생률 높음.', '최근에는 건강검사 통해 개선 중.']},
      {'title': '키우는 난이도',
       'content': ['도시보다는 외곽에서 키우는 게 적합.', '도시에서는 개가 힘들 수 있음.']},
      {'title': '예비 보호자에게',
       'content': ['누구에게나 잘 어울리지만 운동량 확보는 필수.', '산책과 함께 있는 시간이 중요.']},
      {'title': '리트리버 라이프',
       'content': ['가로수길, 홍대 등도 가능.',
        '사람만 있으면 어디든 함께함.',
        '정말 행복한 라이프가 될 수 있음.']},
      {'title': '한마디로 표현', 'content': ['더럽게 착하다.', '인류에게 사랑을 알려주는 견종.']}]}]},
  {'title': '닥스훈트',
   'secti

In [10]:
def flatten_breeds(data):
    docs = []
    for breed in data["breeds"]:
        breed_name = breed["title"]
        for section in breed["sections"]:
            for subtopic in section["subtopics"]:
                topic = subtopic["title"]
                for content in subtopic["content"]:
                    text = f"{breed_name} - {topic}: {content}"
                    docs.append(Document(page_content=text, metadata={"breed": breed_name, "topic": topic}))
    return docs

docs = flatten_breeds(documents)

In [28]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 저장 디렉토리 새로 만들거나 덮어쓰기
vectorstore = Chroma.from_documents(
    docs,
    embedding=embedding,
    persist_directory="./chroma_db",  # 혹시 이전 데이터 남아있으면 덮어쓰기
)

# 저장
vectorstore.persist()


C:\Users\yubny\AppData\Local\Temp\ipykernel_108792\2380837040.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model="text-embedding-3-small")
C:\Users\yubny\AppData\Local\Temp\ipykernel_108792\2380837040.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [30]:
query = "닥스훈트와 골든 리트리버는 어떤 점에서 비슷한 성격이나 특징을 가지고 있나요?"


# vector store 직접 조회(similarity_with_score)
retrievals = vectorstore.similarity_search_with_score(query)
retrievals


[(Document(metadata={'breed': '골든 리트리버', 'topic': '성격'}, page_content='골든 리트리버 - 성격: 침을 많이 흘리는 특성.'),
  1.0027541119514867),
 (Document(metadata={'breed': '닥스훈트', 'topic': '성격'}, page_content='닥스훈트 - 성격: 장난기 많고 에너지가 넘치며, 사냥 본능이 강하다.'),
  1.0405797229879024),
 (Document(metadata={'breed': '닥스훈트', 'topic': '장점'}, page_content='닥스훈트 - 장점: 작고 민첩하며 지능이 높아 다양한 환경에 적응 가능하다.'),
  1.1030253186153487),
 (Document(metadata={'breed': '골든 리트리버', 'topic': '성격'}, page_content='골든 리트리버 - 성격: 도둑도 반기는 경우 있음.'),
  1.1264533033552324)]

In [38]:
# Retriever store 직접 조회
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k':3}
)

retrieval_result = retriever.batch([query])
retrieval_result



[[Document(metadata={'breed': '골든 리트리버', 'topic': '성격'}, page_content='골든 리트리버 - 성격: 침을 많이 흘리는 특성.'),
  Document(metadata={'breed': '닥스훈트', 'topic': '성격'}, page_content='닥스훈트 - 성격: 장난기 많고 에너지가 넘치며, 사냥 본능이 강하다.'),
  Document(metadata={'breed': '닥스훈트', 'topic': '장점'}, page_content='닥스훈트 - 장점: 작고 민첩하며 지능이 높아 다양한 환경에 적응 가능하다.')]]

In [39]:
def search_knowledge(query):
    similar_docs = vectorstore.similarity_search(query, k=3)
    return "\n".join([doc.page_content for doc in similar_docs])

In [40]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate([
    ("system","당신은 반려견 전문가입니다. 아래는 견종 정보입니다. 사용자의 질문에 적절한 견종과 항목을 기반으로 정확하고 친절하게 설명해 주세요."),
    ("user","""
어린이의 질문에 context 만을 이용해 답변하세요.
질문 :{query}
context:{context}
""")
])

In [41]:
prompt.invoke({'query':query,'context':retrievals})

ChatPromptValue(messages=[SystemMessage(content='당신은 반려견 전문가입니다. 아래는 견종 정보입니다. 사용자의 질문에 적절한 견종과 항목을 기반으로 정확하고 친절하게 설명해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content="\n어린이의 질문에 context 만을 이용해 답변하세요.\n질문 :닥스훈트와 골든 리트리버는 어떤 점에서 비슷한 성격이나 특징을 가지고 있나요?\ncontext:[(Document(metadata={'breed': '골든 리트리버', 'topic': '성격'}, page_content='골든 리트리버 - 성격: 침을 많이 흘리는 특성.'), 1.0027541119514867), (Document(metadata={'breed': '닥스훈트', 'topic': '성격'}, page_content='닥스훈트 - 성격: 장난기 많고 에너지가 넘치며, 사냥 본능이 강하다.'), 1.0405797229879024), (Document(metadata={'breed': '닥스훈트', 'topic': '장점'}, page_content='닥스훈트 - 장점: 작고 민첩하며 지능이 높아 다양한 환경에 적응 가능하다.'), 1.1030253186153487), (Document(metadata={'breed': '골든 리트리버', 'topic': '성격'}, page_content='골든 리트리버 - 성격: 도둑도 반기는 경우 있음.'), 1.1264533033552324)]\n", additional_kwargs={}, response_metadata={})])

In [42]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name = 'gpt-4o-mini',
    temperature = 0.5

)

In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#context 생성

query = "리트리버를 키우기 위한 좋은 견주 조건은 뭔가요"
retrievals = retriever.batch([query])
context_text = "\n".join([doc.page_content for doc in retrievals[0]])



chain = prompt | model | StrOutputParser()

In [44]:
chain.invoke({'query':query,'context':retrievals})

'리트리버를 키우기 위한 좋은 견주 조건은 다음과 같아요. 첫째, 리트리버는 활동적인 개이기 때문에 충분한 운동을 시켜줄 수 있어야 해요. 둘째, 물을 좋아하는 성격이 있어서 물가에 데려가 주는 것도 좋답니다. 마지막으로, 리트리버는 훈련을 잘 받는 편이지만, 모든 리트리버가 안내견이 될 수 있는 건 아니기 때문에 그 점도 고려해야 해요. 그래서 리트리버를 잘 키우려면 사랑과 관심을 많이 주는 것이 중요해요!'